<a href="https://colab.research.google.com/github/AlexeyProvorov/Templates/blob/main/Vectors_dependent_and_express_one_as_a_combination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install pyngrok


# Problem

You have three vectors in three-dimensional space:

$$
\vec{a} = \begin{pmatrix} 1 \\\\ 2 \\\\ 3 \end{pmatrix}, \quad
\vec{b} = \begin{pmatrix} 4 \\\\ 5 \\\\ 6 \end{pmatrix}, \quad
\vec{c} = \begin{pmatrix} 7 \\\\ 8 \\\\ 9 \end{pmatrix}
$$

1. **Check whether these vectors are linearly dependent or independent.**
2. **If they are linearly dependent, express one of the vectors as a combination of the other two.**

---

## Hints

1. Vectors are linearly dependent if the determinant of the matrix formed by these vectors as columns is zero (i.e., the matrix is singular).
2. If they are dependent, find coefficients  $\alpha$  and  $\beta$ such that one of the vectors can be written as a linear combination of the other two:

   $$
   \vec{c} = \alpha \vec{a} + \beta \vec{b}
   $$

---

**Try solving it, and let me know if you need any help!**


In [2]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Vectors
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
c = np.array([7, 8, 9])

# Coefficients for linear combination c = -1 * a + 2 * b
alpha = -1
beta = 2
combination = alpha * a + beta * b

# Define the plane passing through vectors a and b
normal_vector = np.cross(a, b)
xx, yy = np.meshgrid(range(-5, 10), range(-5, 10))
z_plane = -(normal_vector[0] * xx + normal_vector[1] * yy) / normal_vector[2]

# Function to add sharp arrows
def add_arrow(fig, start, vec, color, name, row, col, dash="solid", show_marker=False, opacity=1.0, showlegend=True):
    fig.add_trace(go.Scatter3d(
        x=[start[0], vec[0]], y=[start[1], vec[1]], z=[start[2], vec[2]],
        mode='lines', name=name, line=dict(color=color, width=5, dash=dash),
        showlegend=showlegend, opacity=opacity), row=row, col=col)

    arrow_ratio = 0.15
    vec_unit = vec / np.linalg.norm(vec)
    orthogonal_vec1 = np.cross(vec_unit, [0, 0, 1])
    if np.linalg.norm(orthogonal_vec1) < 1e-3:
        orthogonal_vec1 = np.cross(vec_unit, [1, 0, 0])
    orthogonal_vec1 = orthogonal_vec1 / np.linalg.norm(orthogonal_vec1) * arrow_ratio
    orthogonal_vec2 = np.cross(vec_unit, orthogonal_vec1)
    orthogonal_vec2 = orthogonal_vec2 / np.linalg.norm(orthogonal_vec2) * arrow_ratio

    for ov in [orthogonal_vec1, -orthogonal_vec1, orthogonal_vec2, -orthogonal_vec2]:
        fig.add_trace(go.Scatter3d(
            x=[vec[0], vec[0] - arrow_ratio * vec_unit[0] + ov[0]],
            y=[vec[1], vec[1] - arrow_ratio * vec_unit[1] + ov[1]],
            z=[vec[2], vec[2] - arrow_ratio * vec_unit[2] + ov[2]],
            mode='lines', line=dict(color=color, width=5), showlegend=False, opacity=opacity), row=row, col=col)

    if show_marker:
        fig.add_trace(go.Scatter3d(
            x=[start[0]], y=[start[1]], z=[start[2]], mode='markers',
            marker=dict(size=8, color="yellow", symbol="circle-open"), showlegend=False), row=row, col=col)

# Create interactive figures for the original vectors and for steps
fig = make_subplots(rows=1, cols=2, subplot_titles=("Vectors a, b, and c in the Same Plane", "Step-by-Step Construction of c"),
                    specs=[[{"type": "scatter3d"}, {"type": "scatter3d"}]], column_widths=[0.45, 0.45])

# --- Plot 1: Only the original vectors and the common plane ---
add_arrow(fig, [0, 0, 0], a, "blue", "Vector a", row=1, col=1, show_marker=True)
add_arrow(fig, [0, 0, 0], b, "red", "Vector b", row=1, col=1, show_marker=True)
add_arrow(fig, [0, 0, 0], c, "green", "Vector c", row=1, col=1, show_marker=True)

fig.add_trace(go.Surface(
    x=xx, y=yy, z=z_plane, colorscale=[[0, 'LightSkyBlue'], [1, 'LightSkyBlue']],
    showscale=False, opacity=0.9), row=1, col=1)

# --- Plot 2: Step-by-Step Construction ---
add_arrow(fig, [0, 0, 0], -a, "blue", "Step 1: -a", row=1, col=2)
add_arrow(fig, -a, combination, "red", "Step 2: + 2b", row=1, col=2)
add_arrow(fig, [0, 0, 0], c, "green", "Resulting Vector c", row=1, col=2, dash="dash", show_marker=True)

# Adjusting z-values slightly on the xy-plane to create a visible gradient
z_gradient = np.linspace(-0.1, 0.1, xx.shape[1])
z_shifted = np.zeros_like(xx) + z_gradient[np.newaxis, :]

for row, col in [(1, 1), (1, 2)]:
    fig.add_trace(go.Surface(
        x=xx, y=yy, z=z_shifted, colorscale=[[0, 'red'], [0.5, 'white'], [1, 'green']],
        cmin=-0.1, cmax=0.1, showscale=False, opacity=0.1), row=row, col=col)
    fig.add_trace(go.Surface(
        x=xx, y=np.zeros_like(xx), z=yy, colorscale=[[0, 'red'], [0.5, 'white'], [1, 'green']],
        cmin=-10, cmax=10, showscale=False, opacity=0.1), row=row, col=col)
    fig.add_trace(go.Surface(
        x=np.zeros_like(xx), y=xx, z=yy, colorscale=[[0, 'red'], [0.5, 'white'], [1, 'green']],
        cmin=-10, cmax=10, showscale=False, opacity=0.1), row=row, col=col)

# Longer, transparent yellow vectors for x, y, and z axes
axis_length = 9
add_arrow(fig, [0, 0, 0], [axis_length, 0, 0], "yellow", "X Axis", row=1, col=1, opacity=0.2, showlegend=False)
add_arrow(fig, [0, 0, 0], [0, axis_length, 0], "yellow", "Y Axis", row=1, col=1, opacity=0.2, showlegend=False)
add_arrow(fig, [0, 0, 0], [0, 0, axis_length], "yellow", "Z Axis", row=1, col=1, opacity=0.2, showlegend=False)
add_arrow(fig, [0, 0, 0], [axis_length, 0, 0], "yellow", "X Axis", row=1, col=2, opacity=0.2, showlegend=False)
add_arrow(fig, [0, 0, 0], [0, axis_length, 0], "yellow", "Y Axis", row=1, col=2, opacity=0.2, showlegend=False)
add_arrow(fig, [0, 0, 0], [0, 0, axis_length], "yellow", "Z Axis", row=1, col=2, opacity=0.2, showlegend=False)

# General layout settings for both plots
fig.update_layout(
    template="plotly_dark",  # Apply the 'plotly_dark' template
    scene=dict(
        xaxis=dict(nticks=10, range=[-5, 10], showline=False, showticklabels=False, showgrid=False, backgroundcolor="#333333", visible=False),
        yaxis=dict(nticks=10, range=[-5, 10], showline=False, showticklabels=False, showgrid=False, backgroundcolor="#333333", visible=False),
        zaxis=dict(nticks=10, range=[-5, 10], showline=False, showticklabels=False, showgrid=False, backgroundcolor="#333333", visible=False),
        aspectratio=dict(x=1, y=1, z=0.7),
        camera=dict(eye=dict(x=1.25, y=1.25, z=0.75)),
    ),
    scene2=dict(  # Apply the same settings to the second plot
        xaxis=dict(nticks=10, range=[-5, 10], showline=False, showticklabels=False, showgrid=False, backgroundcolor="#333333", visible=False),
        yaxis=dict(nticks=10, range=[-5, 10], showline=False, showticklabels=False, showgrid=False, backgroundcolor="#333333", visible=False),
        zaxis=dict(nticks=10, range=[-5, 10], showline=False, showticklabels=False, showgrid=False, backgroundcolor="#333333", visible=False),
        aspectratio=dict(x=1, y=1, z=0.7),
        camera=dict(eye=dict(x=1.25, y=1.25, z=0.75)),
    ),
    title="Comparison of Vectors and Step-by-Step Construction in 3D Space",
    autosize=False,
    width=1200,
    height=600
)

fig.show()
